In [ ]:
import os
os.chdir('C:\\Github')

In [ ]:
from ctypes import *
import random
from os import listdir
from os.path import isfile, isdir, join
import time
from queue import Queue
from collections import defaultdict
import csv
import pandas as pd
import numpy as np
import ast
import json
from PIL import Image
#import imagehash
import hashlib
import math
from statsmodels.tsa.api import ExponentialSmoothing, Holt, SimpleExpSmoothing

from collections import Counter

In [ ]:
obj_list = ['person','motorbike','backpack','handbag','suitcase',\
'sports ball', 'kite', 'bottle','cup', 'knife', 'bowl',\
'orange', 'pizza', 'chair', 'sofa', 'potted plant', \
'dining table', 'tvmonitor', 'laptop', 'mouse', 'remote', \
'keyboard', 'cell phone', 'microwave', 'oven', 'sink', \
'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'toothbrush']

obj_list_all = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane',\
'bus', 'train', 'truck', 'boat', 'traffic light',\
'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',\
'cat', 'dog', 'horse', 'sheep', 'cow',\
'elephant', 'bear', 'zebra', 'giraffe', 'backpack',\
'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',\
'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',\
'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle',\
'wine glass', 'cup', 'fork', 'knife', 'spoon',\
'bowl', 'banana', 'apple', 'sandwich', 'orange',\
'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',\
'cake', 'chair', 'sofa', 'potted plant', 'bed',\
'dining table', 'toilet', 'tvmonitor', 'laptop',\
'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',\
'oven', 'toaster', 'sink', 'refrigerator', 'book',\
'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',\
'toothbrush']

table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

In [ ]:
def signed_encode(hash_):
    # signed compute
    hash_ = np.where(hash_ > 0, hash_, 0)
    hash_ = np.where(hash_ <= 0, hash_, 1)
    
    return hash_.astype(int)


def image_hash(object_name, weight):
    
    weight = abs(weight)
    
    text_ = hashlib.blake2b(object_name.encode()).hexdigest()
    text_encode = np.array([])
    
    for x in text_:
        value_ = int(x, base=16)
        tmp = '{0:04b}'.format(value_)
        value_list = [int(char) for char in tmp]
        text_encode = np.append(text_encode, value_list)
        text_encode = text_encode.astype(int)

    # weighted
    text_encode = np.where(text_encode < 0, text_encode, text_encode*weight)
    text_encode = np.where(text_encode > 0, text_encode, -1*weight)
    
    return text_encode

def similarity(hash1_, hash2_):
    return 1 - (((abs(hash1_- hash2_)).sum())/len(hash1_))

In [ ]:
def class_to_coordinate(a):
    table = np.array([\
        [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41]])
    
    x = np.argwhere(table == a)[0][1]
    y = np.argwhere(table == a)[0][0]
    
    coordinate = [x,y]
    
    return coordinate

def euclidean_distance(p1,p2):
    x = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,11,11,11,11,11,11,11,11,11,11, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11]
    y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,11,11,11,11,11,11,11,11,11,11]

    x_test_label = x[int(p1)]
    y_test_label = y[int(p1)]
    x_predict_label = x[int(p2)]
    y_predict_label = y[int(p2)]
    error = ((x_test_label - x_predict_label)**2 + (y_test_label - y_predict_label)**2)**0.5
    return error

In [ ]:
#image train Hashing
image_path = f'./Data/0318_92589_train_image/'
image_Train = pd.read_csv(join(image_path, 'image_info.csv'))

list_of_image_Train_hash = []
image_train_label = []
print(len(image_Train))

for i in range(len(image_Train)):
    image_Train_row = image_Train.iloc[i].to_dict()
    image_Train_row_label = image_Train_row['label']
    image_Train_row.pop('label',None)

    # device1 hash encode
    image_Train_hash = np.array([])
    image_Train_list = image_Train.iloc[i].to_list()
    object_count = image_Train_list[4]
    if (object_count != 0) and (object_count < 26):
        for j in range(int(object_count)) : 
            
            object_info = [image_Train_list[(j*6)+5], image_Train_list[(j*6)+6], image_Train_list[(j*6)+7], image_Train_list[(j*6)+8], image_Train_list[(j*6)+9], image_Train_list[(j*6)+10]]

            object_name = obj_list_all[int(object_info[0])]
            weight_count = 2
            weight_area = 20*(object_info[1]*(object_info[4] + object_info[5]))
            weight_x = 20*(0.5 - abs(0.5 - object_info[2]))
            weight_y = 20*(0.5 - abs(0.5 - object_info[3]))

            hash_count = image_hash(f'{object_name}_count', weight_count)
            hash_area = image_hash(f'{object_name}_area', weight_area)
            hash_x = image_hash(f'{object_name}_x', weight_x)
            hash_y = image_hash(f'{object_name}_x', weight_y)

            hash_ = hash_count + hash_area + hash_x + hash_y
            if len(image_Train_hash) < 1:
                image_Train_hash = hash_
            else:
                image_Train_hash = image_Train_hash + hash_

        image_Train_hash = signed_encode(image_Train_hash)
    
        list_of_image_Train_hash.append(image_Train_hash)
        image_train_label.append(image_Train_row_label)

print(len(image_Train))
print(len(image_train_label))
print(len(list_of_image_Train_hash))

In [ ]:
scripted_walk = [
    'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
    'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
]

stationary = [
    'stationary_1'
]

freewalk = [
    'freewalk_1','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_8','freewalk_9'
]


walk_list = freewalk

obj_missing_rate = 1 # 1 8 6
beacon_count = 7  # 7 6 5 4 
beacon_rate = 1
# 1 GOOD 
# 2 1 Bad
# 3 2 Bad

methods_list =  [f'dev_1_A51',f'dev_1_U11',f'dev_1_U19e',f'dev_1_025',\
                 f'dev_2',f'dev_3',f'dev_4'
]

methods = 'dev_4'

free_corner = [31,36,41]

if walk_list == freewalk :
    K = 7
    block_distance_set = 5.0 
    
if walk_list == scripted_walk :
    K = 6
    block_distance_set = 3.0  
    
if walk_list == stationary :
    K = 7
    block_distance_set = 5.0
    
x = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,11,11,11,11,11,11,11,11,11,11, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11]
y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,11,11,11,11,11,11,11,11,11,11]

In [ ]:
if walk_list == freewalk or walk_list == stationary or walk_list == scripted_walk :
    #Image Hashing 一起討論後推舉出候選人投票 binary
    list_MDE = []
    total_DE = 0
    len_of_all_label = 0

    error_distri = [0]*16
    error_distribution = []

    for walk in walk_list:
        print("Walk Method : ",walk)
        hash_similarity = []

        test_label = []
        predict_label = []

        file_name = f'./image_to_pf/92589_itp_{methods}_{walk}_omr_{obj_missing_rate}_bc_{beacon_count}_br_{beacon_rate}.csv'
        with open(file_name, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['label',methods])

            # 讀取 test data
            image_Test = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU19e/image_info.csv'))
            image_Test_device2 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/sharp025/image_info.csv'))
            image_Test_device3 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/GalaxyA51/image_info.csv'))
            image_Test_device4 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU11/image_info.csv'))

            #讀取 wireless_reference_point
            wireless_reference_point_flie = pd.read_csv(f'./wireless_reference_point/{methods}_{walk}_bc_{beacon_count}_br_{beacon_rate}_wireless_reference_point.csv', delimiter=',')
            Pos = []
            list_voter_wireless_reference_point = []

            for index, Row in wireless_reference_point_flie.iterrows():    
                # ground truth
                pos = Row['label']
                wireless_reference_point = Row['wireless_reference_point']
                voter_wireless_reference_point = eval(Row['voter_wireless_reference_point'])
                list_voter_wireless_reference_point.append(Counter(voter_wireless_reference_point))
                Pos.append(pos)

                # video test data setup
                Image_Test = image_Test[image_Test['label'] == pos]
                Image_Test_device2 = image_Test_device2[image_Test_device2['label'] == pos]
                Image_Test_device3 = image_Test_device3[image_Test_device3['label'] == pos]
                Image_Test_device4 = image_Test_device4[image_Test_device4['label'] == pos]

                # choose the first frame
                Image_Test = Image_Test[0:1]
                Image_Test_device2 = Image_Test_device2[0:1]
                Image_Test_device3 = Image_Test_device3[0:1]
                Image_Test_device4 = Image_Test_device4[0:1]

                # 將 test data 轉換成 hashing
            #Device1
                Image_Test_hash = np.array([])
                Image_Test_list = Image_Test.iloc[0].to_list()
                object_count = Image_Test_list[4]
                if (object_count != 0) and (object_count < 26):
                    for j in range(int(object_count)) : 

                        object_info = [Image_Test_list[(j*6)+5], Image_Test_list[(j*6)+6], Image_Test_list[(j*6)+7], Image_Test_list[(j*6)+8], Image_Test_list[(j*6)+9], Image_Test_list[(j*6)+10]]

                        object_name = obj_list_all[int(object_info[0])]
                        weight_count = 2
                        weight_area = 20*(object_info[1]*(object_info[4] + object_info[5]))
                        weight_x = 20*(0.5 - abs(0.5 - object_info[2]))
                        weight_y = 20*(0.5 - abs(0.5 - object_info[3]))

                        hash_count = image_hash(f'{object_name}_count', weight_count)
                        hash_area = image_hash(f'{object_name}_area', weight_area)
                        hash_x = image_hash(f'{object_name}_x', weight_x)
                        hash_y = image_hash(f'{object_name}_x', weight_y)

                        hash_ = hash_count + hash_area + hash_x + hash_y
                        if len(Image_Test_hash) < 1:
                            Image_Test_hash = hash_
                        else:
                            Image_Test_hash = Image_Test_hash + hash_                                       
            #Device1

            #Device2
                Image_Test_hash_device2 = np.array([])
                Image_Test_list_device2 = Image_Test_device2.iloc[0].to_list()
                object_count_device2 = Image_Test_list_device2[4]
                if (object_count_device2 != 0) and (object_count_device2 < 26):
                    for j in range(int(object_count_device2)) : 

                        object_info = [Image_Test_list_device2[(j*6)+5], Image_Test_list_device2[(j*6)+6], Image_Test_list_device2[(j*6)+7], Image_Test_list_device2[(j*6)+8], Image_Test_list_device2[(j*6)+9], Image_Test_list_device2[(j*6)+10]]

                        object_name = obj_list_all[int(object_info[0])]
                        weight_count = 2
                        weight_area = 20*(object_info[1]*(object_info[4] + object_info[5]))
                        weight_x = 20*(0.5 - abs(0.5 - object_info[2]))
                        weight_y = 20*(0.5 - abs(0.5 - object_info[3]))

                        hash_count = image_hash(f'{object_name}_count', weight_count)
                        hash_area = image_hash(f'{object_name}_area', weight_area)
                        hash_x = image_hash(f'{object_name}_x', weight_x)
                        hash_y = image_hash(f'{object_name}_x', weight_y)

                        hash_ = hash_count + hash_area + hash_x + hash_y
                        if len(Image_Test_hash_device2) < 1:
                            Image_Test_hash_device2 = hash_
                        else:
                            Image_Test_hash_device2 = Image_Test_hash_device2 + hash_                                      
            #Device2

            #Device3
                Image_Test_hash_device3 = np.array([])
                Image_Test_list_device3 = Image_Test_device3.iloc[0].to_list()
                object_count_device3 = Image_Test_list_device3[4]
                if (object_count_device3 != 0) and (object_count_device3 < 26):
                    for j in range(int(object_count_device3)) : 

                        object_info = [Image_Test_list_device3[(j*6)+5], Image_Test_list_device3[(j*6)+6], Image_Test_list_device3[(j*6)+7], Image_Test_list_device3[(j*6)+8], Image_Test_list_device3[(j*6)+9], Image_Test_list_device3[(j*6)+10]]

                        object_name = obj_list_all[int(object_info[0])]
                        weight_count = 2
                        weight_area = 20*(object_info[1]*(object_info[4] + object_info[5]))
                        weight_x = 20*(0.5 - abs(0.5 - object_info[2]))
                        weight_y = 20*(0.5 - abs(0.5 - object_info[3]))

                        hash_count = image_hash(f'{object_name}_count', weight_count)
                        hash_area = image_hash(f'{object_name}_area', weight_area)
                        hash_x = image_hash(f'{object_name}_x', weight_x)
                        hash_y = image_hash(f'{object_name}_x', weight_y)

                        hash_ = hash_count + hash_area + hash_x + hash_y
                        if len(Image_Test_hash_device3) < 1:
                            Image_Test_hash_device3 = hash_
                        else:
                            Image_Test_hash_device3 = Image_Test_hash_device3 + hash_                                        
            #Device3

            #Device4
                Image_Test_hash_device4 = np.array([])
                Image_Test_list_device4 = Image_Test_device4.iloc[0].to_list()
                object_count_device4 = Image_Test_list_device4[4]
                if (object_count_device4 != 0) and (object_count_device4 < 26):
                    for j in range(int(object_count_device4)) : 

                        object_info = [Image_Test_list_device4[(j*6)+5], Image_Test_list_device4[(j*6)+6], Image_Test_list_device4[(j*6)+7], Image_Test_list_device4[(j*6)+8], Image_Test_list_device4[(j*6)+9], Image_Test_list_device4[(j*6)+10]]

                        object_name = obj_list_all[int(object_info[0])]
                        weight_count = 2
                        weight_area = 20*(object_info[1]*(object_info[4] + object_info[5]))
                        weight_x = 20*(0.5 - abs(0.5 - object_info[2]))
                        weight_y = 20*(0.5 - abs(0.5 - object_info[3]))

                        hash_count = image_hash(f'{object_name}_count', weight_count)
                        hash_area = image_hash(f'{object_name}_area', weight_area)
                        hash_x = image_hash(f'{object_name}_x', weight_x)
                        hash_y = image_hash(f'{object_name}_x', weight_y)

                        hash_ = hash_count + hash_area + hash_x + hash_y
                        if len(Image_Test_hash_device4) < 1:
                            Image_Test_hash_device4 = hash_
                        else:
                            Image_Test_hash_device4 = Image_Test_hash_device4 + hash_                                     
            #Device4

                Image_Test_hash = signed_encode(Image_Test_hash)
                Image_Test_hash_device2 = signed_encode(Image_Test_hash_device2)
                Image_Test_hash_device3 = signed_encode(Image_Test_hash_device3)
                Image_Test_hash_device4 = signed_encode(Image_Test_hash_device4)

                #找出 Top K 個像的 device1 # 將 train data 與 test data 做比對
                if object_count == 0 :
                    voter = [0.0]*K
                else :
                    k_top_similarity = [0.0]*K
                    voter = [0.0]*K
                    for k in range(len(list_of_image_Train_hash)) :
                        ### Wireless Filtering ###
                        block_distance = euclidean_distance(image_train_label[k],wireless_reference_point)
                        if block_distance < block_distance_set :
                            sim_ = similarity(list_of_image_Train_hash[k], Image_Test_hash)
                            if sim_ > (min(k_top_similarity)) :
                                voter[k_top_similarity.index(min(k_top_similarity))] = image_train_label[k]
                                k_top_similarity[k_top_similarity.index(min(k_top_similarity))] = sim_

                #找出 Top K 個像的 device2
                if object_count_device2 == 0 :
                    voter_device2 = [0.0]*K
                else :
                    k_top_similarity_device2 = [0.0]*K
                    voter_device2 = [0.0]*K
                    for k in range(len(list_of_image_Train_hash)) :
                        block_distance = euclidean_distance(image_train_label[k],wireless_reference_point)
                        if block_distance < block_distance_set :
                            sim_ = similarity(list_of_image_Train_hash[k], Image_Test_hash_device2)
                            if sim_ > (min(k_top_similarity_device2)) :
                                voter_device2[k_top_similarity_device2.index(min(k_top_similarity_device2))] = image_train_label[k]
                                k_top_similarity_device2[k_top_similarity_device2.index(min(k_top_similarity_device2))] = sim_

                #找出 Top K 個像的 device3
                if object_count_device3 == 0 :
                    voter_device3 = [0.0]*K
                else :
                    k_top_similarity_device3 = [0.0]*K
                    voter_device3 = [0.0]*K
                    for k in range(len(list_of_image_Train_hash)) :
                        block_distance = euclidean_distance(image_train_label[k],wireless_reference_point)
                        if block_distance < block_distance_set :
                            sim_ = similarity(list_of_image_Train_hash[k], Image_Test_hash_device3)
                            if sim_ > (min(k_top_similarity_device3)) :
                                voter_device3[k_top_similarity_device3.index(min(k_top_similarity_device3))] = image_train_label[k]
                                k_top_similarity_device3[k_top_similarity_device3.index(min(k_top_similarity_device3))] = sim_

                #找出 Top K 個像的 device4
                if object_count_device4 == 0 :
                    voter_device4 = [0.0]*K
                else :
                    k_top_similarity_device4 = [0.0]*K
                    voter_device4 = [0.0]*K
                    for k in range(len(list_of_image_Train_hash)) :
                        block_distance = euclidean_distance(image_train_label[k],wireless_reference_point)
                        if block_distance < block_distance_set :
                            sim_ = similarity(list_of_image_Train_hash[k], Image_Test_hash_device4)
                            if sim_ > (min(k_top_similarity_device4)) :
                                voter_device4[k_top_similarity_device4.index(min(k_top_similarity_device4))] = image_train_label[k]
                                k_top_similarity_device4[k_top_similarity_device4.index(min(k_top_similarity_device4))] = sim_
                                

                print("Ground Truth :" , pos)
                print("Wireless Point :",float(wireless_reference_point))

                voter_of_predict = []
                # 根據物件數量做加權
                for m in range(len(voter)):
                    for n in range(int(object_count)):            # u19e
                        voter_of_predict.append(voter[m])

                    for o in range(int(object_count_device2)):    # 025
                        voter_of_predict.append(voter_device2[m])

                    for p in range(int(object_count_device3)):    # a51
                        voter_of_predict.append(voter_device3[m])

                    for q in range(int(object_count_device4)):    # u11
                        voter_of_predict.append(voter_device4[m])

                ### Dynamic Weighting ###
                if (walk_list == freewalk) and (wireless_reference_point in free_corner) and (beacon_count == 7) :
                    voter_of_predict = [float(wireless_reference_point)]*len(voter_of_predict)

                if (walk_list == stationary) and (wireless_reference_point in free_corner) and (beacon_count == 7) :
                    voter_of_predict = [float(wireless_reference_point)]*len(voter_of_predict)

                if len(voter_of_predict) == 0 :
                    voter_of_predict = [float(wireless_reference_point)]
                ### Dynamic Weighting ###

                predict_label.append(max(voter_of_predict, key = voter_of_predict.count))
                print("predict_label :",max(voter_of_predict, key = voter_of_predict.count))
                print("--")

                ### Dynamic Weighting ###
                if beacon_count == 7 :
                    for vwrp in range(len(voter_wireless_reference_point)) :
                        voter_of_predict.append(voter_wireless_reference_point[vwrp])
                if beacon_count == 6 :
                    for vwrp in range(30) :
                        voter_of_predict.append(voter_wireless_reference_point[vwrp])
                if beacon_count == 5 :
                    for vwrp in range(15) :
                        voter_of_predict.append(voter_wireless_reference_point[vwrp])
                if beacon_count == 4 :
                    for vwrp in range(5) :
                        voter_of_predict.append(voter_wireless_reference_point[vwrp])
                ### Dynamic Weighting ###
                
                writer.writerow([pos,voter_of_predict])

    #計算 MDE
        test_label = Pos
        MDE = 0

        for i in range(len(test_label)):
            x_test_label = x[int(test_label[i])]
            y_test_label = y[int(test_label[i])]
            x_predict_label = x[int(predict_label[i])]
            y_predict_label = y[int(predict_label[i])]
            error = ((x_test_label - x_predict_label)**2 + (y_test_label - y_predict_label)**2)**0.5
            error_distri[math.ceil(error)] += 1
            MDE += error
            error_distribution.append(error*0.6)

        total_DE += MDE*0.6
        len_of_all_label += len(test_label)
        MDE = MDE*0.6/len(test_label)
        MDE = round(MDE, 4)
        list_MDE.append(MDE)

        print(methods,"in" ,walk," Mean Distance Error is : ", MDE)    
    #計算 MDE      

    print(list_MDE)
    print(walk_list)
    MDE = total_DE/len_of_all_label
    print("Mean Distance Error of ",methods,"in " ,walk,"is : ",MDE)
    print(len_of_all_label)
    print(error_distri)